In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import functools
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2024-01-31'
final_date = '2024-01-31'

# READING FILES

### EDO DE CUENTA

In [3]:
all_mov = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\CONCILIACIONES\2024\01. ENERO\EDO CTA BNTE\*.txt', recursive=False)
all_mov

['C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT1.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT10.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT11.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT12.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT13.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT14.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT15.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\01. ENERO\\EDO CTA BNTE\\BNTE 2024.01.02 PT16.txt',
 'C:\\Users\\Cristian.Aparicio\\Documents\\CONCILIACIONES\\2024\\

# EDO CUENTA COMPLETO

In [4]:
edo_cta = pd.concat([pd.read_csv(f,sep='|',dtype={'Cuenta': object,'Cod. Transac':object,'Referencia':object})
                     .assign(file_name=os.path.basename(f)) for f in all_mov])
edo_cta.reset_index(inplace=True)
del edo_cta['index']
edo_cta.sort_values(by=['Movimiento'],inplace=True)
edo_cta['Cuenta'] = "'"+edo_cta['Cuenta']
edo_cta['ID'] = ''
edo_cta['TYPE'] = ''
edo_cta['Descripción'].fillna('SIN DESCRIPCION',inplace=True)
edo_cta.head()

,Cuenta,Fecha De Operación,Fecha,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,file_name,ID,TYPE
0,'0239650517,30/12/2023,02/01/2024,0300001,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,$934.28,"$501,321,740.73",41507057,"CTA/CLABE:002760701768361110 ,CB SPEI, BCO: 00...",NaN,BNTE 2024.01.02 PT1.txt,,
1,'0239650517,30/12/2023,02/01/2024,0363373,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,$635.32,"$501,321,105.41",41507058,"CTA/CLABE:014700230013272347 ,CB SPEI, BCO: 01...",NaN,BNTE 2024.01.02 PT1.txt,,
2,'0239650517,30/12/2023,02/01/2024,0331988,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,"$2,108.14","$501,318,997.27",41507059,"CTA/CLABE:012180004693203856 ,CB SPEI, BCO: 01...",NaN,BNTE 2024.01.02 PT1.txt,,
3,'0239650517,30/12/2023,02/01/2024,0300006,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,$527.03,"$501,318,470.24",41507060,"CTA/CLABE:044010256029105948 ,CB SPEI, BCO: 04...",NaN,BNTE 2024.01.02 PT1.txt,,
4,'0239650517,30/12/2023,02/01/2024,0363378,COMPRA ORDEN DE PAGO SPEI,511,4264,$0.00,"$2,801.91","$501,315,668.33",41507061,"CTA/CLABE:021694064625716414 ,CB SPEI, BCO: 02...",NaN,BNTE 2024.01.02 PT1.txt,,


In [5]:
edo_cta.shape

(1230196, 16)

In [6]:
duplicates_edo_cta = edo_cta[edo_cta.duplicated(['Movimiento'] , keep=False)]
duplicates_edo_cta.head()

,Cuenta,Fecha De Operación,Fecha,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,file_name,ID,TYPE


In [7]:
duplicates_edo_cta.shape

(0, 16)

In [8]:
split_fecha_operacion = edo_cta["Fecha De Operación"].str.split("/",n=3,expand=True)
split_fecha_operacion.head()

,0,1,2
0,30,12,2023
1,30,12,2023
2,30,12,2023
3,30,12,2023
4,30,12,2023


In [9]:
edo_cta["Day_operacion"] = split_fecha_operacion[0] 
edo_cta["Month_operacion"] = split_fecha_operacion[1]
edo_cta['Year_operacion'] = split_fecha_operacion[2]

In [10]:
split_fecha_banco = edo_cta["Fecha"].str.split("/",n=3,expand=True)
split_fecha_banco.head()

,0,1,2
0,02,01,2024
1,02,01,2024
2,02,01,2024
3,02,01,2024
4,02,01,2024


In [11]:
edo_cta["Day_banco"] = split_fecha_banco[0] 
edo_cta["Month_banco"] = split_fecha_banco[1]
edo_cta['Year_banco'] = split_fecha_banco[2]

In [12]:
edo_cta['Fecha_operacion'] = edo_cta['Year_operacion']+'-'+edo_cta['Month_operacion']+'-'+edo_cta['Day_operacion']
edo_cta['Fecha_banco'] = edo_cta['Year_banco']+'-'+edo_cta['Month_banco']+'-'+edo_cta['Day_banco']

In [13]:
edo_cta['Fecha_operacion'] = pd.to_datetime(edo_cta['Fecha_operacion'])
edo_cta['Fecha_banco'] = pd.to_datetime(edo_cta['Fecha_banco'])

### AFILIACIONES AGREGADOR

In [14]:
searchfor = ['07668673',
'07927479',
'07927482',
'07927484',
'07927489',
'07927492',
'07927496',
'07927500',
'07927504',
'07927514',
'07927515',
'07927516',
'07927517',
'07927518',
'07927519',
'07927520',
'08382645',
'08382648']

In [15]:
edo_cta.loc[edo_cta['Descripción'].str.contains('08343798'), "Tipo"] = 'XPAY'
edo_cta.loc[edo_cta['Descripción'].str.contains('08672604'), "Tipo"] = 'XPAY'
edo_cta.loc[edo_cta['Descripción'].str.contains('08380402'), "Tipo"] = 'MIT-3DS'
edo_cta.loc[edo_cta['Descripción'].str.contains('|'.join(searchfor)), "Tipo"] = 'REGULAR TXN'
edo_cta.loc[(~edo_cta['Descripción'].str.contains('08343798'))
            & (~edo_cta['Descripción'].str.contains('08380402'))
            & (~edo_cta['Descripción'].str.contains('08672604'))
            & (~edo_cta['Descripción'].str.contains('|'.join(searchfor))), "Tipo"] = 'TASA_NATURAL'

In [16]:
# edo_cta.loc[edo_cta['Descripción'].str.contains('08343798'), "Tipo"] = 'XPAY'
# edo_cta.loc[edo_cta['Descripción'].str.contains('08380402'), "Tipo"] = 'MIT-3DS'
# edo_cta.loc[edo_cta['Descripción'].str.contains('08672604'), "Tipo"] = 'XPAY'
# edo_cta.loc[edo_cta['Descripción'].str.contains('08548664'), "Tipo"] = 'TASA_NATURAL'
# edo_cta.loc[edo_cta['Descripción'].str.contains('08536735'), "Tipo"] = 'TASA_NATURAL'
# edo_cta.loc[(~edo_cta['Descripción'].str.contains('08343798')) & (~edo_cta['Descripción'].str.contains('08380402'))
#             & (~edo_cta['Descripción'].str.contains('08548668')) & (~edo_cta['Descripción'].str.contains('08548664'))
#             & (~edo_cta['Descripción'].str.contains('08536735')) , "Tipo"] = 'REGULAR TXN'


In [17]:
edo_cta.Depósitos = [x.strip('$') for x in edo_cta.Depósitos]
edo_cta.Retiros = [x.strip('$') for x in edo_cta.Retiros]
# edo_cta.Saldo = [x.strip('(') for x in edo_cta.Saldo]
# edo_cta.Saldo = [x.strip(')') for x in edo_cta.Saldo]
edo_cta.Saldo = [x.strip('$') for x in edo_cta.Saldo]

In [18]:
edo_cta['Depósitos'] = edo_cta['Depósitos'].str.replace(',','').astype(float)
edo_cta['Retiros'] = edo_cta['Retiros'].str.replace(',','').astype(float)
# edo_cta['Saldo'] = edo_cta['Saldo'].str.replace(',','').astype(float)

In [19]:
edo_cta.loc[edo_cta['Cod. Transac'] == '060', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '061', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '062', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '600', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '601', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[edo_cta['Cod. Transac'] == '602', 'Afiliacion_1'] = edo_cta['Descripción'].str[-9:]
edo_cta.loc[(edo_cta['Cod. Transac'] == '602') & (edo_cta['Descripción Detallada'].str.contains('CONTRACARGO')), 'Afiliacion_1'] = edo_cta['Descripción'].str[-8:]
edo_cta['len'] = edo_cta['Afiliacion_1'].str.len() 
edo_cta.loc[edo_cta['len'] == 9, 'Afiliacion_final'] = edo_cta['Afiliacion_1'].str[-8:-1]
edo_cta.loc[edo_cta['len'] == 8, 'Afiliacion_final'] = edo_cta['Afiliacion_1'].str[-7:]
edo_cta.loc[edo_cta['len'] == 9, 'Tipo_operacion'] = edo_cta['Afiliacion_1'].str[-1:]
edo_cta.loc[edo_cta['Afiliacion_1'].isnull(), 'Tipo'] = ''
edo_cta.head()

,Cuenta,Fecha De Operación,Fecha,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,...,Day_banco,Month_banco,Year_banco,Fecha_operacion,Fecha_banco,Tipo,Afiliacion_1,len,Afiliacion_final,Tipo_operacion
0,'0239650517,30/12/2023,02/01/2024,0300001,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,934.28,"501,321,740.73",...,02,01,2024,2023-12-30,2024-01-02,,NaN,NaN,NaN,NaN
1,'0239650517,30/12/2023,02/01/2024,0363373,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,635.32,"501,321,105.41",...,02,01,2024,2023-12-30,2024-01-02,,NaN,NaN,NaN,NaN
2,'0239650517,30/12/2023,02/01/2024,0331988,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,"2,108.14","501,318,997.27",...,02,01,2024,2023-12-30,2024-01-02,,NaN,NaN,NaN,NaN
3,'0239650517,30/12/2023,02/01/2024,0300006,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,527.03,"501,318,470.24",...,02,01,2024,2023-12-30,2024-01-02,,NaN,NaN,NaN,NaN
4,'0239650517,30/12/2023,02/01/2024,0363378,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,"2,801.91","501,315,668.33",...,02,01,2024,2023-12-30,2024-01-02,,NaN,NaN,NaN,NaN


In [20]:
edo_cta.shape

(1230196, 29)

In [21]:
all_edo = edo_cta[['Cuenta','Fecha_operacion','Fecha_banco','Referencia','Descripción','Cod. Transac','Sucursal','Depósitos'
                   ,'Retiros','Saldo','Movimiento','Descripción Detallada','Cheque','Tipo','Afiliacion_1','Afiliacion_final'
                   ,'Tipo_operacion']]
all_edo.head()

,Cuenta,Fecha_operacion,Fecha_banco,Referencia,Descripción,Cod. Transac,Sucursal,Depósitos,Retiros,Saldo,Movimiento,Descripción Detallada,Cheque,Tipo,Afiliacion_1,Afiliacion_final,Tipo_operacion
0,'0239650517,2023-12-30,2024-01-02,0300001,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,934.28,"501,321,740.73",41507057,"CTA/CLABE:002760701768361110 ,CB SPEI, BCO: 00...",NaN,,NaN,NaN,NaN
1,'0239650517,2023-12-30,2024-01-02,0363373,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,635.32,"501,321,105.41",41507058,"CTA/CLABE:014700230013272347 ,CB SPEI, BCO: 01...",NaN,,NaN,NaN,NaN
2,'0239650517,2023-12-30,2024-01-02,0331988,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,"2,108.14","501,318,997.27",41507059,"CTA/CLABE:012180004693203856 ,CB SPEI, BCO: 01...",NaN,,NaN,NaN,NaN
3,'0239650517,2023-12-30,2024-01-02,0300006,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,527.03,"501,318,470.24",41507060,"CTA/CLABE:044010256029105948 ,CB SPEI, BCO: 04...",NaN,,NaN,NaN,NaN
4,'0239650517,2023-12-30,2024-01-02,0363378,COMPRA ORDEN DE PAGO SPEI,511,4264,0.00,"2,801.91","501,315,668.33",41507061,"CTA/CLABE:021694064625716414 ,CB SPEI, BCO: 02...",NaN,,NaN,NaN,NaN


In [22]:
all_edo.shape

(1230196, 17)

In [23]:
all_edo_1 = all_edo[(all_edo['Fecha_operacion'] >= '2024-01-01') & (all_edo['Fecha_operacion'] <= '2024-01-20')]
all_edo_1['Fecha_operacion'].unique()

<DatetimeArray>
['2024-01-01 00:00:00', '2024-01-02 00:00:00', '2024-01-03 00:00:00',
 '2024-01-04 00:00:00', '2024-01-05 00:00:00', '2024-01-06 00:00:00',
 '2024-01-07 00:00:00', '2024-01-08 00:00:00', '2024-01-09 00:00:00',
 '2024-01-10 00:00:00', '2024-01-11 00:00:00', '2024-01-12 00:00:00',
 '2024-01-13 00:00:00', '2024-01-14 00:00:00', '2024-01-15 00:00:00',
 '2024-01-16 00:00:00', '2024-01-17 00:00:00', '2024-01-18 00:00:00',
 '2024-01-19 00:00:00', '2024-01-20 00:00:00']
Length: 20, dtype: datetime64[ns]

In [24]:
all_edo_1.shape

(656462, 17)

In [25]:
all_edo_2 = all_edo[(all_edo['Fecha_operacion'] >= '2024-01-21') & (all_edo['Fecha_operacion'] <= '2024-01-31')]
all_edo_2['Fecha_operacion'].unique()

<DatetimeArray>
['2024-01-21 00:00:00', '2024-01-22 00:00:00', '2024-01-23 00:00:00',
 '2024-01-24 00:00:00', '2024-01-25 00:00:00', '2024-01-26 00:00:00',
 '2024-01-27 00:00:00', '2024-01-28 00:00:00', '2024-01-29 00:00:00',
 '2024-01-30 00:00:00', '2024-01-31 00:00:00']
Length: 11, dtype: datetime64[ns]

In [26]:
all_edo_2.shape

(354938, 17)

# ESTADO DE CUENTA ALL

In [27]:
writer = pd.ExcelWriter('EDO CUENTA DICIEMBRE 2023.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
# all_edo.to_excel(writer, sheet_name='2023.08.01 - 2023.08.31',index=False)
all_edo_1.to_excel(writer, sheet_name='2023.12.01 - 2023-12-20',index=False)
all_edo_2.to_excel(writer, sheet_name='2023.12.21 - 2023-12-31',index=False)
writer.close()

In [28]:
# all_edo.to_excel('EDO CUENTA ABRIL 2023.xlsx',index=False)

# ESTADO DE CUENTA POR SEMANA

In [29]:
edo_cta_weekly = edo_cta[(edo_cta['Fecha_operacion'] >= start_date) & (edo_cta['Fecha_operacion'] <= final_date)]
edo_cta_weekly['Fecha_operacion'].unique()

<DatetimeArray>
['2024-01-31 00:00:00']
Length: 1, dtype: datetime64[ns]

In [30]:
edo_cta_weekly_final = edo_cta_weekly[['ID','Cuenta','Fecha_operacion','Fecha_banco','Referencia','Descripción','TYPE'
                                       ,'Cod. Transac','Sucursal','Depósitos','Retiros','Saldo','Movimiento'
                                       ,'Descripción Detallada','Cheque','Tipo','Afiliacion_1','Afiliacion_final'
                                       ,'Tipo_operacion']]
edo_cta_weekly_final.fillna('nan',inplace=True)
edo_cta_weekly_final['ID'] = 'nan'
edo_cta_weekly_final['TYPE'] = 'nan'
edo_cta_weekly_final['limit'] = 'nan'
edo_cta_weekly_final['dev_column'] = 'nan'
edo_cta_weekly_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_19688\2641352024.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  edo_cta_weekly_final.fillna('nan',inplace=True)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_19688\2641352024.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edo_cta_weekly_final.fillna('nan',inplace=True)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_19688\2641352024.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,ID,Cuenta,Fecha_operacion,Fecha_banco,Referencia,Descripción,TYPE,Cod. Transac,Sucursal,Depósitos,...,Saldo,Movimiento,Descripción Detallada,Cheque,Tipo,Afiliacion_1,Afiliacion_final,Tipo_operacion,limit,dev_column
1228505,nan,'0239650517,2024-01-31,2024-01-31,0020240130,COMISION X OPERACION CB,nan,537,4264,0.00,...,"139,056,242.95",42735562,"OPERACIONES CONEXION BANORTE, OP 02: 000000001...",nan,,nan,nan,nan,nan,nan
1228506,nan,'0239650517,2024-01-31,2024-01-31,0000000000,IVA DE COMISION X OPERACION CB,nan,517,4264,0.00,...,"139,056,210.15",42735563,nan,nan,,nan,nan,nan,nan,nan
1228507,nan,'0239650517,2024-01-31,2024-01-31,0000000000,CLIP MX*KASSEL HOME 08584266D,nan,060,302,0.01,...,"139,056,210.16",42735564,nan,nan,TASA_NATURAL,08584266D,8584266,D,nan,nan
1228508,nan,'0239650517,2024-01-31,2024-01-31,0000000000,CLIPMX*AGREGADOR 07927482C,nan,060,302,"33,124.36",...,"139,089,334.52",42735565,No. de Pagos 12 Tipo de Plan 03 Periodo...,nan,REGULAR TXN,07927482C,7927482,C,nan,nan
1228509,nan,'0239650517,2024-01-31,2024-01-31,0000000000,COMISION 07927482C,nan,600,302,0.00,...,"139,088,721.72",42735566,No. de Pagos 12 Tipo de Plan 03 Periodo...,nan,REGULAR TXN,07927482C,7927482,C,nan,nan


In [31]:
edo_cta_weekly_final.shape

(1691, 21)

### CONCILIACION

In [32]:
edo_cta_weekly_final['Depósitos'].unique()

array([0.00000000e+00, 1.00000000e-02, 3.31243600e+04, 1.52500000e+04,
       1.96088000e+04, 1.10782240e+05, 6.36032600e+05, 3.93480090e+05,
       1.55000000e+03, 5.45680000e+04, 4.01090690e+05, 1.19494960e+05,
       4.92254210e+05, 5.93526050e+05, 4.06334150e+06, 2.89406128e+06,
       1.69050000e+04, 3.98589400e+04, 2.03384680e+05, 1.30680830e+05,
       3.59332220e+05, 3.29151990e+05, 1.26619759e+07, 2.32038750e+07,
       3.41371510e+05, 5.00870000e+04, 1.12233400e+05, 5.49973800e+04,
       1.08588589e+06, 5.72279260e+05, 2.57102300e+04, 3.24134000e+04,
       5.47552000e+04, 1.57500000e+04, 6.73500000e+03, 2.45300000e+03,
       3.81100000e+03, 5.00000000e+02, 2.87000000e+03, 7.00000000e+03,
       1.39312000e+03, 2.28400000e+03, 4.92300000e+03, 1.09178200e+04,
       3.07604500e+04, 4.82784500e+04, 1.56786100e+04, 2.98000000e+03,
       3.00000000e+02, 3.00000000e+03, 2.87800000e+03, 2.22456000e+04,
       6.42296400e+04, 8.12872670e+05, 3.34789630e+05, 1.16955182e+06,
      

In [33]:
edo_cta_weekly_final['Depósitos'].dtypes

dtype('float64')

In [34]:
edo_cta_weekly_final['Retiros'].dtypes

dtype('float64')

In [35]:
edo_cta_weekly_final['Saldo'].dtypes

dtype('O')

In [36]:
edo_cta_weekly_final['TYPE'] = edo_cta_weekly_final['Descripción Detallada'].str[13:15].replace(' ', '',regex=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_19688\392704393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edo_cta_weekly_final['TYPE'] = edo_cta_weekly_final['Descripción Detallada'].str[13:15].replace(' ', '',regex=True)


In [37]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['TYPE'] == '', 'TYPE'] = edo_cta_weekly_final['Descripción'].str[29:30].replace(' ', '',regex=True)

In [38]:
 edo_cta_weekly_final['TYPE'].unique()

array(['ON', 'B', 'D', '12', '09', '06', '03', 'C', '24', '18', ',', 'PL',
       '00', '45', '80', '15', '02', '10', '76', '58', '88', '85', '69',
       '19', '32', '68', '59', '05', '29', '66', '91', '43', '81', '27',
       '54', '70', '16', 'CT', '40', '04'], dtype=object)

In [39]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Depósitos'] != 0, 'ID'] = 1
edo_cta_weekly_final.loc[(edo_cta_weekly_final['Descripción'].str.contains('IVA COMISION') & (edo_cta_weekly_final['ID'] != 1)), 'ID'] = 2
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str[:8] == 'COMISION' , 'ID'] = 1
edo_cta_weekly_final.loc[edo_cta_weekly_final['ID'] == '' , 'ID'] = 'nan'

In [40]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('COMISION ORDEN DE PAGO SPEI'), 'ID'] = 11
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('I.V.A. ORDEN DE PAGO SPEI') , 'ID'] = 22
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('COMPRA ORDEN DE PAGO SPEI') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('TRASPASO MERCHANT'), 'ID'] = 'FS'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('TRASPASO BANAMEX MSI'), 'TYPE'] = 'BAN'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('TIPO DE PLAN 20') &
                         edo_cta_weekly_final['Cod. Transac'] == '060', 'TYPE'] = 'PLAN20'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('COMPRA ORDEN DE PAGO SPEI') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('TRASPASO MERCHANT'), 'TYPE'] = 'nan'

In [41]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('AMERICAN EXPRESS'), 'TYPE'] = 'AMEX'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('OPENPAY'), 'TYPE'] = 'OP'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Cod. Transac'] == '602', 'TYPE'] = 'CC'
edo_cta_weekly_final['Referencia'] = 0


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_19688\3673321521.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edo_cta_weekly_final['Referencia'] = 0


In [42]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Cod. Transac'] == '504', 'TYPE'] = 'H2H'
edo_cta_weekly_final.loc[(edo_cta_weekly_final['Cod. Transac'] == '511') & 
                         ~edo_cta_weekly_final['Descripción Detallada'].str.contains('=REFERENCIA'), 'TYPE'] = 'H2H'
edo_cta_weekly_final.loc[(edo_cta_weekly_final['Cod. Transac'] == '001'), 'TYPE'] = 'DEV H2H'
edo_cta_weekly_final.loc[(edo_cta_weekly_final['Cod. Transac'] == '003') & 
                          edo_cta_weekly_final['Descripción'].str.contains('RETORNO SPEI'), 'TYPE'] = 'DEV H2H'
edo_cta_weekly_final.loc[(edo_cta_weekly_final['Cod. Transac'] == '003') & 
                          edo_cta_weekly_final['Descripción'].str.contains('DEV'), 'TYPE'] = 'DEV H2H'
edo_cta_weekly_final.loc[(edo_cta_weekly_final['Cod. Transac'] == '003') & 
                          edo_cta_weekly_final['Descripción Detallada'].str.contains('ANUL SPEI'), 'TYPE'] = 'DEV H2H'

In [43]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('COMISION X OPERACION') , 'TYPE'] = 'COMCB'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('IVA DE COMISION X OPERACION') , 'TYPE'] = 'IVACOMCB'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('COM CONEXI') , 'TYPE'] = 'COMCB'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('IVA COMISION CXB') , 'TYPE'] = 'IVACOMCB'

In [44]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Tipo'].str.contains('XPAY') &
                         edo_cta_weekly_final['Tipo_operacion'].str.contains('D'), 'TYPE'] = 'ECOMD'

In [45]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Tipo'].str.contains('XPAY') &
                         edo_cta_weekly_final['Tipo_operacion'].str.contains('C'), 'TYPE'] = 'ECOMC'

In [46]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Tipo'].str.contains('TASA_NATURAL') &
                         edo_cta_weekly_final['Tipo_operacion'].str.contains('D'), 'TYPE'] = 'TNATD'

In [47]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Tipo'].str.contains('TASA_NATURAL') &
                         edo_cta_weekly_final['Tipo_operacion'].str.contains('C'), 'TYPE'] = 'TNATC'

In [48]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('TARJ No') &
                         ((edo_cta_weekly_final['Descripción'].str.contains('CLIP MX')) |
                          (edo_cta_weekly_final['Descripción'].str.contains('PAYCLIP')) |
                         (edo_cta_weekly_final['Descripción'].str.contains('COMISION')) |
                         (edo_cta_weekly_final['Descripción'].str.contains('IVA COMISION'))), 'TYPE'] = edo_cta_weekly_final['Tipo_operacion']

In [49]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Tipo'].str.contains('XPAY') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('CONTRACARGO'), 'TYPE'] = 'CC'

In [50]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('TRASPASO A CUENTA PROPIA') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('4204'), 'TYPE'] = 'TRAS4204'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('TRASPASO A CUENTA PROPIA') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('7640'), 'TYPE'] = 'TRAS7640'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('TRASPASO A CUENTA PROPIA') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('7490'), 'TYPE'] = 'TRAS7490'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('DEPOSITO DE CUENTA PROPIA') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('4204'), 'TYPE'] = 'CTA4204'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('DEPOSITO DE CUENTA PROPIA') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('7640'), 'TYPE'] = 'CTA7640'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('DEPOSITO DE CUENTA PROPIA') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('9335'), 'TYPE'] = 'CTA9335'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('DEPOSITO DE CUENTA PROPIA') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('7490'), 'TYPE'] = 'CTA7490'

In [51]:

edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('REV VTA'), 'dev_column'] = 'DEV'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('REV CLIP'), 'dev_column'] = 'DEV'
edo_cta_weekly_final.loc[((edo_cta_weekly_final['Descripción'].str.contains('CLIP MX')) &
                         (edo_cta_weekly_final['Depósitos'] == '0.00') &
                          (~edo_cta_weekly_final['Descripción Detallada'].str.contains('CONTRACARGO'))), 'dev_column'] = 'DEV'
edo_cta_weekly_final.loc[((edo_cta_weekly_final['Descripción'].str.contains('CLIPCOM')) &
                         (edo_cta_weekly_final['Depósitos'] == '0.00') &
                          (~edo_cta_weekly_final['Descripción Detallada'].str.contains('CONTRACARGO'))), 'dev_column'] = 'DEV'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('COMISION') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('COMISION'), 'dev_column'] = 'DEV'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('IVA') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('COMISION'), 'dev_column'] = 'DEV'

In [52]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('COMISION ENVIO MT940'), 'TYPE'] = 'MT940'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('IVA DE COMISION ENVIO MT940'), 'TYPE'] = 'IVAMT940'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('COMISION ENVIO MT940'), 'dev_column'] = ''
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('IVA DE COMISION ENVIO MT940'), 'dev_column'] = ''

In [53]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('COBRO VISA'), 'TYPE'] = 'COBROVISA'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].isin(['CERTVISA','CERTIFICACISN VISA']), 'TYPE'] = 'CERTVISA'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('NONCOMPLIANCEFEE'), 'TYPE'] = 'NONCOMPFEE'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('INVERSION MDD'), 'TYPE'] = 'INV'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('VENC. CAP. MDD'), 'TYPE'] = 'VENCCAPMDD'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('INTERES MDD'), 'TYPE'] = 'INTMDD'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('Impuesto Sobre la Renta'), 'TYPE'] = 'ISR'

In [54]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('DE LA CLABE 014180655095310419'), 'TYPE'] = 'TRAS0419'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('DE LA CLABE 012180001953705822'), 'TYPE'] = 'CTA582'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('DE LA CLABE 110180000776461644'), 'TYPE'] = 'CTA164'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('DE LA CLABE 002180700815061818'), 'TYPE'] = 'CTA1818'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('DE LA CLABE 002180700475932617'), 'TYPE'] = 'CTA2617'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('DE LA CLABE 014180655068329767'), 'TYPE'] = 'CTA9767'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('DE LA CLABE 014180655081499454'), 'TYPE'] = 'DEP SWAP'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('DE LA CLABE 646180202711900005'), 'TYPE'] = 'DEP SWAP'

In [55]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Cod. Transac'].str.contains('003') &
                         edo_cta_weekly_final['Descripción Detallada'].str.contains('CACAO PAYCARD SOLUTIONS'), 'TYPE'] = 'DEVCACAO'

In [56]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción'].str.contains('CC CORRECCION CARGO'), 'TYPE'] = 'COR.CARGO'

In [57]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('REVERSO CONTRACARGO'), 'TYPE'] = 'RCC'

In [58]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('COM.ESTADO CTA'), 'TYPE'] = 'COMEDOCTA'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('IVA COM.EDO.CTA'), 'TYPE'] = 'IVACOMEDOCTA'

In [59]:
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('COMPENSACION SPEI'), 'TYPE'] = 'COMPSPEI'
edo_cta_weekly_final.loc[edo_cta_weekly_final['Descripción Detallada'].str.contains('COMPENSACION DESFASE'), 'TYPE'] = 'COMPSPEI'

In [60]:
edo_cta_weekly_final['limit'] = edo_cta_weekly_final.groupby('Fecha_operacion')['Cuenta'].shift(-1)
edo_cta_weekly_final['limit'].fillna('LIMIT',inplace=True)
edo_cta_weekly_final['limit'].replace("'0239650517",'',inplace=True)


C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_19688\2531087657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edo_cta_weekly_final['limit'] = edo_cta_weekly_final.groupby('Fecha_operacion')['Cuenta'].shift(-1)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_19688\2531087657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edo_cta_weekly_final['limit'].fillna('LIMIT',inplace=True)
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_19688\2531087657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [61]:
edo_cta_weekly_final.replace('nan','',inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_19688\934645789.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edo_cta_weekly_final.replace('nan','',inplace=True)


In [62]:
edo_cta_weekly_final.head()

,ID,Cuenta,Fecha_operacion,Fecha_banco,Referencia,Descripción,TYPE,Cod. Transac,Sucursal,Depósitos,...,Saldo,Movimiento,Descripción Detallada,Cheque,Tipo,Afiliacion_1,Afiliacion_final,Tipo_operacion,limit,dev_column
1228505,1,'0239650517,2024-01-31,2024-01-31,0,COMISION X OPERACION CB,COMCB,537,4264,0.00,...,"139,056,242.95",42735562,"OPERACIONES CONEXION BANORTE, OP 02: 000000001...",,,,,,,
1228506,,'0239650517,2024-01-31,2024-01-31,0,IVA DE COMISION X OPERACION CB,IVACOMCB,517,4264,0.00,...,"139,056,210.15",42735563,,,,,,,,
1228507,1,'0239650517,2024-01-31,2024-01-31,0,CLIP MX*KASSEL HOME 08584266D,TNATD,060,302,0.01,...,"139,056,210.16",42735564,,,TASA_NATURAL,08584266D,8584266,D,,
1228508,1,'0239650517,2024-01-31,2024-01-31,0,CLIPMX*AGREGADOR 07927482C,12,060,302,"33,124.36",...,"139,089,334.52",42735565,No. de Pagos 12 Tipo de Plan 03 Periodo...,,REGULAR TXN,07927482C,7927482,C,,
1228509,1,'0239650517,2024-01-31,2024-01-31,0,COMISION 07927482C,12,600,302,0.00,...,"139,088,721.72",42735566,No. de Pagos 12 Tipo de Plan 03 Periodo...,,REGULAR TXN,07927482C,7927482,C,,


In [63]:
edo_cta_weekly_final.shape

(1691, 21)

In [64]:
edo_cta_weekly_final['TYPE'].unique()

array(['COMCB', 'IVACOMCB', 'TNATD', '12', '09', '06', '03', 'C', 'D',
       '24', '18', 'TNATC', 'ECOMC', 'ECOMD', 'AMEX', 'H2H', 'CC',
       'CTA9767', 'DEV H2H', 'CT', '40', 'CTA4204', 'TRAS0419'],
      dtype=object)

In [65]:
edo_cta_weekly_final['ID'].unique()

array([1, '', 2, 11, 22], dtype=object)

In [66]:
edo_cta_weekly_final.to_csv('EDO CUENTA ENERO 31.csv',index=False)